## <span style="color:#db7d60">Setup</span>

In [1]:
%load_ext autoreload
%autoreload 2

import json
import math
import pickle
import numpy as np
import pandas as pd
import textwrap
import sys
from pathlib import Path
from IPython.display import display, Markdown
import matplotlib.pyplot as plt

# pUtil needs to be loaded from the root directory
parent_dir = Path().resolve().parent
sys.path.insert(0, str(parent_dir))

import pUtil
from analysis_common import *
import data_manager
import analysis as anal

import vector
from particle import Particle
from particle import PDGID

plt.rcParams['figure.figsize'] = (21, 6)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

/global/homes/e/eshy/.local/lib/python3.11/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


## <span style="color:#db7d60">Analysis</span>

In [ ]:
anal.plotting.plot_training_run(['model_10M_9_exp9_vdi_2',
                                 'model_exp10_2M_lr1',
                                 'model_exp10_5M_lr1',
                                 'model_exp10_500k_lr1',
                                 'model_exp10_100k_lr1'], use_log=False, x_lim=(0, 70000))

anal.plotting.plot_training_run(['model_10M_9_exp9_vdi_2',
                                 'model_exp10_2M_lr1',
                                 'model_exp10_5M_lr1',
                                 'model_exp10_500k_lr1',
                                 'model_exp10_100k_lr1'], use_log=False, x_lim=(0, 6000))

anal.plotting.plot_training_run(['model_10M_9_exp9_vdi_2',
                                 'model_exp10_2M_lr1',
                                 'model_exp10_5M_lr1',
                                 'model_exp10_500k_lr1',
                                 'model_exp10_100k_lr1'], use_log=False, y_lim=(4.35, 4.65), x_lim=(0, 70000))

anal.plotting.plot_training_run(['model_10M_9_exp9_vdi_2',
                                 'model_exp10_2M_lr1',
                                 'model_exp10_5M_lr1',
                                 'model_exp10_500k_lr1',
                                 'model_exp10_100k_lr1'], use_log=False, y_lim=(4.35, 4.65), x_lim=(0, 6000))

In [4]:
df = anal.tables.get_default_df([
    'model_10M_9_exp9_vdi_2',
    'model_exp10_2M_lr1',
    'model_exp10_5M_lr1',
    'model_exp10_500k_lr1',
    'model_exp10_100k_lr1'])

display(df[['model_name', 'iters_saved', 'min_saved_train_loss', 'min_saved_val_loss'] + anal.tables.model_config_columns])
display(df[['model_name', 'iters_saved', 'min_saved_train_loss', 'min_saved_val_loss'] + anal.tables.model_config_columns].sort_values(by='min_saved_val_loss', ascending=True))

,model_name,iters_saved,min_saved_train_loss,min_saved_val_loss,batch_size,block_size,learning_rate,min_lr,lr_decay_iters,lr_scheduler,n_layer,n_head,n_embd,scheme,preparation_name
0,model_10M_9_exp9_vdi_2,38000,4.471382,4.481986,128,NaN,0.001,0.000001,45000,cosine_annealing_with_warmup,4,2,512,unknown,unknown
1,model_exp10_2M_lr1,10500,4.478233,4.537992,128,NaN,0.001,0.000001,45000,cosine_annealing_with_warmup,4,2,512,unknown,unknown
2,model_exp10_5M_lr1,33000,4.465014,4.492705,128,NaN,0.001,0.000001,45000,cosine_annealing_with_warmup,4,2,512,unknown,unknown
3,model_exp10_500k_lr1,1500,4.566786,4.630126,128,NaN,0.001,0.000001,45000,cosine_annealing_with_warmup,4,2,512,unknown,unknown
4,model_exp10_100k_lr1,500,4.593980,4.562336,128,NaN,0.001,0.000001,45000,cosine_annealing_with_warmup,4,2,512,unknown,unknown


,model_name,iters_saved,min_saved_train_loss,min_saved_val_loss,batch_size,block_size,learning_rate,min_lr,lr_decay_iters,lr_scheduler,n_layer,n_head,n_embd,scheme,preparation_name
0,model_10M_9_exp9_vdi_2,38000,4.471382,4.481986,128,NaN,0.001,0.000001,45000,cosine_annealing_with_warmup,4,2,512,unknown,unknown
2,model_exp10_5M_lr1,33000,4.465014,4.492705,128,NaN,0.001,0.000001,45000,cosine_annealing_with_warmup,4,2,512,unknown,unknown
1,model_exp10_2M_lr1,10500,4.478233,4.537992,128,NaN,0.001,0.000001,45000,cosine_annealing_with_warmup,4,2,512,unknown,unknown
4,model_exp10_100k_lr1,500,4.593980,4.562336,128,NaN,0.001,0.000001,45000,cosine_annealing_with_warmup,4,2,512,unknown,unknown
3,model_exp10_500k_lr1,1500,4.566786,4.630126,128,NaN,0.001,0.000001,45000,cosine_annealing_with_warmup,4,2,512,unknown,unknown
